In [1]:
import zipfile
import os
import numpy as np
import csv
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
pasta_origem = r'C:\Users\Pedro Carvalho\Desktop\Quanti\Arquivos_Diarios'
pasta_destino = r'C:\Users\Pedro Carvalho\Desktop\Arquivos_Diarios_Novo'  

if not os.path.exists(pasta_destino):
    os.makedirs(pasta_destino)

for arquivo in os.listdir(pasta_origem):
    if arquivo.endswith('.zip'):
        caminho_zip = os.path.join(pasta_origem, arquivo)
        with zipfile.ZipFile(caminho_zip, 'r') as zip_ref:
            zip_ref.extractall(pasta_destino)

In [3]:
arquivos_extraidos = os.listdir(pasta_destino)

arquivos_csv = [arquivo for arquivo in arquivos_extraidos if arquivo.endswith('.csv')]

for arquivo_csv in arquivos_csv:
    caminho_csv = os.path.join(pasta_destino, arquivo_csv)
    df = pd.read_csv(caminho_csv)
    print(f"Lendo arquivo: {arquivo_csv}")
    print(df.head())  

Lendo arquivo: Quanti_20240601.csv
           ISIN      AuM Fund AuM Fund Date  AuM Share Class  \
0  LU0602537226  1.004964e+08    2024-05-31      19313180.07   
1  IE00B3Q9WN89  1.844495e+08    2024-04-30      25532354.87   
2  LU0529382367  1.025686e+08    2024-05-30      56805261.72   
3  LU0622426806  4.472222e+08    2024-05-31      44199702.31   
4  LU0624261797  4.472222e+08    2024-05-31      22473526.00   

  AuM Share Class Date  Bid NAV Bid NAV Date  Ask NAV Ask NAV Date  \
0           2024-05-31      NaN          NaN      NaN          NaN   
1           2022-01-31      NaN          NaN      NaN          NaN   
2           2024-05-30      NaN          NaN      NaN          NaN   
3           2024-05-31      NaN          NaN      NaN          NaN   
4           2024-05-31      NaN          NaN      NaN          NaN   

   Valuation NAV Valuation NAV Date  Transaction NAV Transaction NAV Date  \
0        13.2173         2024-05-31              NaN                  NaN   
1    

In [4]:
lista_dfs = []

for arquivo_csv in arquivos_csv:
    caminho_csv = os.path.join(pasta_destino, arquivo_csv)
    df = pd.read_csv(caminho_csv)
    lista_dfs.append(df)

df_diarios_final = pd.concat(lista_dfs, ignore_index=True)

In [6]:
def get_data_e_valor(row):
    if pd.notna(row['FE Bid']):
        return pd.Series([row['FE Bid Date'], row['FE Bid']])
    elif pd.notna(row['Valuation NAV']): 
        return pd.Series([row['Valuation NAV Date'], row['Valuation NAV']])
    elif pd.notna(row['FE Generic NAV']):
        return pd.Series([row['FE Generic NAV Date'], row['FE Generic NAV']])
    elif pd.notna(row['FE NAV']):
        return pd.Series([row['FE NAV Date'], row['FE NAV']])
    elif pd.notna(row['Transaction NAV']):
        return pd.Series([row['Transaction NAV Date'], row['Transaction NAV']])
    else:
        return pd.Series([np.nan, np.nan])

df_diarios_final[['data', 'valor']] = df_diarios_final.apply(get_data_e_valor, axis=1)

df_diarios_final_amostragem = df_diarios_final[['ISIN', 'data', 'valor']].dropna().sort_values(by=['ISIN', 'data'], ascending=[True, True]).head(250)

print(df_diarios_final_amostragem)

                 ISIN        data    valor
85857    990000002459  2024-05-29  254.660
207340   990000002459  2024-05-30  255.540
308181   990000002459  2024-05-31  255.960
414235   990000002459  2024-06-03  256.120
519739   990000002459  2024-06-04  255.020
620768   990000002459  2024-06-05  257.360
750025   990000002459  2024-06-06  258.340
851389   990000002459  2024-06-07  257.360
954813   990000002459  2024-06-10  254.380
1058611  990000002459  2024-06-11  251.830
1163337  990000002459  2024-06-12  254.200
1285099  990000002459  2024-06-13  250.130
1385333  990000002459  2024-06-14  244.730
1482120  990000002459  2024-06-17  246.690
1573430  990000002459  2024-06-18  248.020
1673606  990000002459  2024-06-19  246.710
1796313  990000002459  2024-06-20  249.560
1897358  990000002459  2024-06-21  248.160
2003236  990000002459  2024-06-24  250.330
2107772  990000002459  2024-06-25  249.100
2212197  990000002459  2024-06-26  247.810
67108    990000004939  2024-05-29    6.450
190430   99

In [8]:
# Leitura do arquivo CSV
df = pd.read_csv(r'C:\Users\Pedro Carvalho\Desktop\Quanti\Lote_Inicial\LoteInicial\LoteInicial.csv')

df_sorted = df.sort_values(by=['ISIN', 'General Reference Date',], ascending=[True, True]).tail(250)

print(df_sorted)

MemoryError: Unable to allocate 1.30 GiB for an array with shape (174668777,) and data type object